<a href="https://colab.research.google.com/github/DongIn-Kim/StockResearch/blob/main/stockdatahandling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PyMySQL

     |████████████████████████████████| 51kB 2.7MB/s 


In [3]:
import pymysql
import pandas as pd

In [4]:
db = pymysql.connect(host='sr-aws-1.cc9lwciyheos.ap-northeast-2.rds.amazonaws.com',
            port=3306,
            user='root', 
            password='1q2w3e4r!', 
            db='Investar',
            charset='utf8mb4')
cursor = db.cursor()

In [15]:
sql = "SELECT * FROM daily_price;" 
cursor.execute(sql)
result = cursor.fetchall()
daily_price = pd.DataFrame(result)
daily_price


,0,1,2,3,4,5,6,7
0,000020,2016-12-26,7840,7950,7840,7930,100,41707
1,000020,2016-12-27,7930,7980,7880,7900,30,78257
2,000020,2016-12-28,7950,8140,7850,8130,230,99508
3,000020,2016-12-29,8000,8130,7990,8120,10,48540
4,000020,2017-01-02,8160,8170,8060,8110,10,60571
...,...,...,...,...,...,...,...,...
2176588,950200,2021-01-18,14300,14650,13800,13850,450,267479
2176589,950200,2021-01-19,13850,14100,13250,13700,150,367365
2176590,950200,2021-01-20,13950,14400,13650,14250,550,270526
2176591,950200,2021-01-21,14450,14450,13700,13850,400,231236


In [14]:
sql = "SELECT * FROM company_info;;" 
cursor.execute(sql)
result = cursor.fetchall()
company_info = pd.DataFrame(result)
company_info


,0,1,2
0,000020,동화약품,2021-01-24
1,000040,KR모터스,2021-01-24
2,000050,경방,2021-01-24
3,000060,메리츠화재,2021-01-24
4,000070,삼양홀딩스,2021-01-24
...,...,...,...
2407,950160,코오롱티슈진,2021-01-24
2408,950170,JTC,2021-01-24
2409,950180,SNK,2021-01-24
2410,950190,미투젠,2021-01-24


In [17]:
daily_price.columns=['code','date','open','high','low','close','diff','volume']
company_info.columns=['code','company','last_update']

In [18]:
df = pd.merge(company_info,daily_price, left_on='code', right_on='code', how='left')
df.head()

,code,company,last_update,date,open,high,low,close,diff,volume
0,000020,동화약품,2021-01-24,2016-12-26,7840.0,7950.0,7840.0,7930.0,100.0,41707.0
1,000020,동화약품,2021-01-24,2016-12-27,7930.0,7980.0,7880.0,7900.0,30.0,78257.0
2,000020,동화약품,2021-01-24,2016-12-28,7950.0,8140.0,7850.0,8130.0,230.0,99508.0
3,000020,동화약품,2021-01-24,2016-12-29,8000.0,8130.0,7990.0,8120.0,10.0,48540.0
4,000020,동화약품,2021-01-24,2017-01-02,8160.0,8170.0,8060.0,8110.0,10.0,60571.0


In [28]:
df['lag1'] = df.groupby('code')['close'].shift(1)
df['logr'] = (df['close'] / df['lag1'])
df.head()

,code,company,last_update,date,open,high,low,close,diff,volume,lag1,logr
0,000020,동화약품,2021-01-24,2016-12-26,7840.0,7950.0,7840.0,7930.0,100.0,41707.0,NaN,NaN
1,000020,동화약품,2021-01-24,2016-12-27,7930.0,7980.0,7880.0,7900.0,30.0,78257.0,7930.0,0.996217
2,000020,동화약품,2021-01-24,2016-12-28,7950.0,8140.0,7850.0,8130.0,230.0,99508.0,7900.0,1.029114
3,000020,동화약품,2021-01-24,2016-12-29,8000.0,8130.0,7990.0,8120.0,10.0,48540.0,8130.0,0.998770
4,000020,동화약품,2021-01-24,2017-01-02,8160.0,8170.0,8060.0,8110.0,10.0,60571.0,8120.0,0.998768


In [34]:
meanstd = df.groupby(['code'])["logr"].agg(["mean",'std'])
meanstd.head()

,mean,std
code,,
000020,1.001214,0.031823
000040,1.002530,0.097531
000050,1.000068,0.023381
000060,1.000146,0.019114
000070,1.000035,0.025777
